# Advent of Code 2024: Day 15
https://adventofcode.com/2024/day/15


## Part 1
Find the final position of every box after being pushed around by a robot

In [1]:
with open("input.txt", "r") as f:
    data = f.read()


def make_map(data):
    data_map = []
    data_instructions = []
    m = True
    for line in data.split("\n"):
        if not line:
            m = False
        else:
            if m:
                data_map.append([c for c in line])
            else:
                data_instructions.extend([c for c in line])
    return data_map, data_instructions


data_map, data_instructions = make_map(data)

MAX_Y = len(data_map)
MAX_X = len(data_map[0])
DIRS = {
    ">": (0, 1),
    "<": (0, -1),
    "v": (1, 0),
    "^": (-1, 0),
}
position = tuple[int, int]


def get_pos_by_value(data: list[list[str]], value: str) -> list[tuple[int, int] | None]:
    starts = []
    for i in range(MAX_Y):
        for j in range(MAX_X):
            if data[i][j] == value:
                starts.append((i, j))
    return starts


def _update_position(pos: position, dir: tuple[int, int]) -> position:
    return tuple(y + x for y, x in zip(pos, dir))


def _move(data: list[list[str]], pos: position, dir: tuple[int, int]):
    new_y, new_x = _update_position(pos, dir)
    if data[new_y][new_x] == ".":
        data[new_y][new_x] = "@"
        data[pos[0]][pos[1]] = "."
        return data, (new_y, new_x)
    elif data[new_y][new_x] == "O":
        tmp_y = new_y
        tmp_x = new_x
        while True:
            tmp_y, tmp_x = _update_position((tmp_y, tmp_x), dir)
            if data[tmp_y][tmp_x] == "#":
                return data, pos
            if data[tmp_y][tmp_x] == ".":
                if tmp_y < pos[0] or tmp_x < pos[1]:
                    for i in range(tmp_y, pos[0]):
                        s = data[i][tmp_x]
                        data[i][tmp_x] = data[i + 1][tmp_x]
                        data[i + 1][tmp_x] = s
                    for i in range(tmp_x, pos[1]):
                        s = data[tmp_y][i]
                        data[tmp_y][i] = data[tmp_y][i + 1]
                        data[tmp_y][i + 1] = s
                else:
                    for i in range(tmp_y, pos[0], -1):
                        s = data[i][tmp_x]
                        data[i][tmp_x] = data[i - 1][tmp_x]
                        data[i - 1][tmp_x] = s
                    for i in range(tmp_x, pos[1], -1):
                        s = data[tmp_y][i]
                        data[tmp_y][i] = data[tmp_y][i - 1]
                        data[tmp_y][i - 1] = s
                return data, (new_y, new_x)
    return data, pos


pos = get_pos_by_value(data_map, "@")[0]
for dir in data_instructions:
    data_map, pos = _move(data_map, pos, DIRS[dir])

total = 0
for i in range(MAX_Y):
    for j in range(MAX_X):
        if data_map[i][j] == "O":
            total += (i * 100) + j

print(total)

1478649


## Part 2
Find the final position of every box (now wider) after being pushed around by a robot

In [2]:
def make_wide_map(data):
    data_map = []
    data_instructions = []
    m = True
    for line in data.split("\n"):
        if not line:
            m = False
        else:
            if m:
                row = []
                for c in line:
                    if c == "#":
                        row.extend(["#", "#"])
                    elif c == ".":
                        row.extend([".", "."])
                    elif c == "O":
                        row.extend(["[", "]"])
                    elif c == "@":
                        row.extend(["@", "."])
                data_map.append(row)
            else:
                data_instructions.extend([c for c in line])
    return data_map, data_instructions


data_map, data_instructions = make_wide_map(data)
MAX_Y = len(data_map)
MAX_X = len(data_map[0])


def _move(data: list[list[str]], pos: position, dir: tuple[int, int]):
    new_y, new_x = _update_position(pos, dir)
    if data[new_y][new_x] == ".":
        data[new_y][new_x] = "@"
        data[pos[0]][pos[1]] = "."
        return data, (new_y, new_x)
    elif data[new_y][new_x] == "[" or data[new_y][new_x] == "]":
        tmp_y = new_y
        tmp_x = new_x
        if dir == DIRS["<"] or dir == DIRS[">"]:
            while True:
                tmp_y, tmp_x = _update_position((tmp_y, tmp_x), dir)
                if data[tmp_y][tmp_x] == "#":
                    return data, pos
                if data[tmp_y][tmp_x] == ".":
                    if tmp_y < pos[0] or tmp_x < pos[1]:
                        for i in range(tmp_y, pos[0]):
                            s = data[i][tmp_x]
                            data[i][tmp_x] = data[i + 1][tmp_x]
                            data[i + 1][tmp_x] = s
                        for i in range(tmp_x, pos[1]):
                            s = data[tmp_y][i]
                            data[tmp_y][i] = data[tmp_y][i + 1]
                            data[tmp_y][i + 1] = s
                    else:
                        for i in range(tmp_y, pos[0], -1):
                            s = data[i][tmp_x]
                            data[i][tmp_x] = data[i - 1][tmp_x]
                            data[i - 1][tmp_x] = s
                        for i in range(tmp_x, pos[1], -1):
                            s = data[tmp_y][i]
                            data[tmp_y][i] = data[tmp_y][i - 1]
                            data[tmp_y][i - 1] = s
                    return data, (new_y, new_x)
        elif dir == DIRS["^"]:
            ps = set((pos, (tmp_y, tmp_x)))
            xs = set((tmp_x,))
            tmp_s = data[tmp_y][tmp_x]
            if tmp_s == "[":
                ps.add((tmp_y, tmp_x + 1))
                xs.add(tmp_x + 1)
            elif tmp_s == "]":
                ps.add((tmp_y, tmp_x - 1))
                xs.add(tmp_x - 1)
            while True:
                n_pos = len(ps)
                tmp_ps = [
                    _update_position((tmp_y, x), dir) for x in xs if (tmp_y, x) in ps
                ]
                tmp_y -= 1
                for tmp_p in tmp_ps:
                    if data[tmp_p[0]][tmp_p[1]] == "#":
                        return data, pos
                    elif data[tmp_p[0]][tmp_p[1]] == "[":
                        if data[tmp_p[0] + 1][tmp_p[1]] == "[":
                            ps.add(tmp_p)
                        elif data[tmp_p[0] + 1][tmp_p[1]] == "]":
                            ps.add(tmp_p)
                            ps.add((tmp_p[0], tmp_p[1] + 1))
                            xs.add(tmp_p[1] + 1)
                    elif data[tmp_p[0]][tmp_p[1]] == "]":
                        if data[tmp_p[0] + 1][tmp_p[1]] == "]":
                            ps.add(tmp_p)
                        elif data[tmp_p[0] + 1][tmp_p[1]] == "[":
                            ps.add(tmp_p)
                            ps.add((tmp_p[0], tmp_p[1] - 1))
                            xs.add(tmp_p[1] - 1)
                if len(ps) == n_pos:
                    ps_list = list(ps)
                    ps_list.sort(key=lambda x: x[0])
                    for p in ps_list:
                        s = data[p[0]][p[1]]
                        data[p[0]][p[1]] = data[p[0] - 1][p[1]]
                        data[p[0] - 1][p[1]] = s
                    return data, (new_y, new_x)
                elif len(ps) == n_pos + 1:
                    return data, pos
        elif dir == DIRS["v"]:
            ps = set((pos, (tmp_y, tmp_x)))
            xs = set((tmp_x,))
            tmp_s = data[tmp_y][tmp_x]
            if tmp_s == "[":
                ps.add((tmp_y, tmp_x + 1))
                xs.add(tmp_x + 1)
            elif tmp_s == "]":
                ps.add((tmp_y, tmp_x - 1))
                xs.add(tmp_x - 1)
            while True:
                n_pos = len(ps)
                tmp_ps = [
                    _update_position((tmp_y, x), dir) for x in xs if (tmp_y, x) in ps
                ]
                tmp_y += 1
                for tmp_p in tmp_ps:
                    if data[tmp_p[0]][tmp_p[1]] == "#":
                        return data, pos
                    elif data[tmp_p[0]][tmp_p[1]] == "[":
                        if data[tmp_p[0] - 1][tmp_p[1]] == "[":
                            ps.add(tmp_p)
                        elif data[tmp_p[0] - 1][tmp_p[1]] == "]":
                            ps.add(tmp_p)
                            ps.add((tmp_p[0], tmp_p[1] + 1))
                            xs.add(tmp_p[1] + 1)
                    elif data[tmp_p[0]][tmp_p[1]] == "]":
                        if data[tmp_p[0] - 1][tmp_p[1]] == "]":
                            ps.add(tmp_p)
                        elif data[tmp_p[0] - 1][tmp_p[1]] == "[":
                            ps.add(tmp_p)
                            ps.add((tmp_p[0], tmp_p[1] - 1))
                            xs.add(tmp_p[1] - 1)
                if len(ps) == n_pos:
                    ps_list = list(ps)
                    ps_list.sort(key=lambda x: x[0], reverse=True)
                    for p in ps_list:
                        s = data[p[0]][p[1]]
                        data[p[0]][p[1]] = data[p[0] + 1][p[1]]
                        data[p[0] + 1][p[1]] = s
                    return data, (new_y, new_x)
                elif len(ps) == n_pos + 1:
                    return data, pos
    return data, pos


pos = get_pos_by_value(data_map, "@")[0]
for dir in data_instructions:
    data_map, pos = _move(data_map, pos, DIRS[dir])

total = 0
for i in range(MAX_Y):
    for j in range(MAX_X):
        if data_map[i][j] == "[":
            total += (i * 100) + j

print(total)

1495455
